### Making label imgs with `msk`

In [5]:
import struct
import codecs
import os
import re
import sys
import time

from PyQt4.QtGui import *
from PyQt4.QtCore import *
import numpy as np
from PIL import Image

In [23]:
def save_label_as_npy(source_folder_path, msk_file_name, save_folder_path):
        print "Processing " + msk_file_name + " ..."

        if not os.path.isfile(source_folder_path + '/' + msk_file_name) :
            print "Error : There's no file '%s'."%(fileDir + '/' + maskFileName)
            return -1, None

        f = open(source_folder_path + '/' + msk_file_name,'rb')
        fData = f.read()
        fResX, fResY, numMaskLayer = struct.unpack('iii', fData[0:12])

        totalSize = fResY * fResX
        bufSize = totalSize/32
        if totalSize%32 :
            bufSize += 1


        index = 3
        labelArray = np.zeros((fResY, fResX), dtype=int)
        for i in range(numMaskLayer) :
            bitPos = 1
            unitInfo = int(struct.unpack('I', fData[index*4:(index+1)*4])[0])
            for y in range(fResY) :
                for x in range(fResX) :                    
                    if (unitInfo & bitPos) :
                        labelArray[y, x] = 1
                        
                    if bitPos == 0x80000000 :
                        bitPos = 1
                        index+=1
                        if ((index)<bufSize*numMaskLayer+3) :
                            unitInfo = int(struct.unpack('I', fData[index*4:(index+1)*4])[0])
                    else :
                        bitPos = bitPos << 1
            
            if (bitPos!=1) :
                index+=1
        
        f.close()

        filename_without_ext = msk_file_name.split(".")[0]
        np.save(save_folder_path + "/numpy/" + filename_without_ext, labelArray)
        labelImg = Image.fromarray((labelArray * 255).astype(np.uint8))
        labelImg.save(save_folder_path + "/jpg/" + filename_without_ext + ".jpg")

In [24]:
source_folder_path = "./DATA/ORG/msk/"
save_folder_path = "./DATA/LABELED"

file_list = os.listdir(source_folder_path)
file_list_msk = [file for file in file_list if file.endswith(".msk")]

In [25]:
already_processed_files = set(os.listdir(save_folder_path + "/numpy/"))

for file_name in file_list_msk:
    numpy_file_name = file_name.replace(file_name.split(".")[-1], "npy")
    if not numpy_file_name in already_processed_files:
        save_label_as_npy(source_folder_path, file_name, save_folder_path)
    else:
        print(file_name + " is already processed")

0014_f_58_12_lip_puckerer_tex.msk is already processed
0011_f_45_06_jaw_right_tex.msk is already processed
0004_f_20_03_mouth_stretch_tex.msk is already processed
0002_m_24_07_jaw_forward_tex.msk is already processed
0009_f_23_06_jaw_right_tex.msk is already processed
0014_f_58_04_anger_tex.msk is already processed
0014_f_58_18_eye_closed_tex.msk is already processed
0008_f_25_04_anger_tex.msk is already processed
0010_f_22_07_jaw_forward_tex.msk is already processed
0013_m_20_08_mouth_left_tex.msk is already processed
0011_f_45_07_jaw_forward_tex.msk is already processed
0009_f_23_15_lip_roll_tex.msk is already processed
0005_f_45_13_lip_funneler_tex.msk is already processed
0006_m_24_05_jaw_left_tex.msk is already processed
0010_f_22_12_lip_puckerer_tex.msk is already processed
0001_f_22_02_smile_tex.msk is already processed
0010_f_22_16_grin_tex.msk is already processed
0017_m_31_07_jaw_forward_tex.msk is already processed
0013_m_20_19_brow_raiser_tex.msk is already processed
0001_f

error: unpack requires a string argument of length 12